In [4]:
import os
import carputils.carpio.igb as igb
import numpy as np

In [5]:
os.chdir(".")
filename = "../data/only_fiber_conductivity/vm.igb"

In [6]:
data = igb.read(filename)
data[1]

{'x': 191568,
 'y': 1,
 'z': 1,
 't': 501,
 'type': 'float',
 'systeme': 'little_endian',
 'dim_x': 1.0,
 'dim_t': 500.0,
 'inc_t': 1.0,
 'unites_x': 'um',
 'unites_y': 'um',
 'unites_z': 'um',
 'unites_t': 'ms',
 'unites': 'mV'}

In [8]:
Vm = data[0]
Vm.shape

(191568, 501)

In [26]:
def read_points(pts_file):
    """
    Read the .pts file and return an array of points with dimension (n_points x 3).
    """
    print(f"Reading points from {pts_file}")
    
    with open(pts_file, 'r') as f:
        # Read the number of points from the first line
        n_points = int(f.readline().strip())
        
        # Initialize the points array
        points = np.zeros((n_points, 3))
        
        # Read each point
        for i in range(n_points):
            line = f.readline().strip()
            # Parse x, y, z coordinates
            x, y, z = map(float, line.split())
            points[i] = [x, y, z]
    
    print(f"Loaded {n_points} points with shape {points.shape}")
    return points

points = read_points("../data/instance_001_lowres/instance_001_lowres.pts")

Reading points from ../data/instance_001_lowres/instance_001_lowres.pts
Loaded 191568 points with shape (191568, 3)


In [29]:
i_max = 501
dyn_pts = np.zeros([i_max] + list(points.shape))
for i in range(0, i_max):
    t = np.pi * i / i_max
    scale = 1 - 0.1 * np.sin(t)**2.0
    dyn_pts[i] = scale * points

In [30]:
import numpy as np
from carputils.carpio import igb

# Create a small test dataset
data = dyn_pts.astype(np.float32)
print(data.shape)
# Write the file with all necessary header fields
file = igb.IGBFile("../data/test.dynpt", mode="w")
file.write(data, header={
    "type": "vec3f",
    "t": dyn_pts.shape[0],
    "x": dyn_pts.shape[1],
    "inc_t": 0.5,
    "dim_t": (dyn_pts.shape[0] - 1) * 0.5,
})
file.close()

(501, 191568, 3)
